In [2]:
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/My Drive/data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
pip install transformers

     |████████████████████████████████| 542kB 2.8MB/s 
     |████████████████████████████████| 870kB 51.4MB/s 
     |████████████████████████████████| 1.0MB 44.8MB/s 
     |████████████████████████████████| 3.7MB 38.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=9ff8adff1f9e400d8299060491d43e31e822d720cff81568dbe46f2a051ca0b8
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [24]:
pip install textstat

     |████████████████████████████████| 3.0MB 4.1MB/s 


In [25]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import os
import pandas.io.json as pd_json
from nltk.corpus import stopwords
from collections import Counter,defaultdict
import string
import textstat
import statistics
import pickle
import torch
from transformers import *
from sklearn.feature_extraction.text import TfidfVectorizer

In [29]:
#HOUSEKEEPING
stop_words = stopwords.words('english')
punct_removal_table = {ord(char): None for char in string.punctuation}

MODELS = {"BertModel" : (BertModel,       BertTokenizer,       'bert-base-uncased'),
          "OpenAIGPTModel" : (OpenAIGPTModel,  OpenAIGPTTokenizer,  'openai-gpt'),
          "GPT2Model" : (GPT2Model,       GPT2Tokenizer,       'gpt2'),
          "TransfoXLModel" : (TransfoXLModel,  TransfoXLTokenizer,  'transfo-xl-wt103'),
          "XLNetModel" : (XLNetModel,      XLNetTokenizer,      'xlnet-base-cased'),
          "XLMModel" : (XLMModel,        XLMTokenizer,        'xlm-mlm-enfr-1024'),
          "DistilBertModel" : (DistilBertModel, DistilBertTokenizer, 'distilbert-base-cased'),
          "RobertaModel" : (RobertaModel,    RobertaTokenizer,    'roberta-base'),
          "XLMRobertaModel" : (XLMRobertaModel, XLMRobertaTokenizer, 'xlm-roberta-base')}
model_class, tokenizer_class, pretrained_weights = MODELS['BertModel']
#Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)
nlp = pipeline('feature-extraction') 

#read author-affiliation file from csrankings hidden page 
author_university_df = pd.read_csv(os.path.join(data_path,"author_university_list.csv"))
#read university-research score file from csrankings main page
university_score_df = pd.read_csv(os.path.join(data_path,"csrankings.csv")).drop_duplicates(subset=['institute'])
'''with open("processed_data/affiliation_dict", "rb") as input_file:
  email_institute_affiliation_mapper = pickle.load(input_file)'''
email_institute_affiliation_mapper = build_affiliation_dictionary(author_university_df,university_score_df)

In [10]:
def load_data_files_into_raw_df(tfidf_matrix):

    directory_in_string = data_path + '/iclr_2017/train/parsed_pdfs/304.pdf.json'
    directory_content = os.fsencode(directory_in_string)
    paper_data_df = pd.DataFrame()
    file_number = 0
    for file in os.listdir(directory_content):
        filename = os.fsdecode(file)
        file_dict = defaultdict()
        with open(os.path.join(directory_in_string, filename),encoding="utf8") as file:
            #df = pd.read_json(file,orient='columns')
            data = json.load(file)
            paper_id = data['name']
            paper_metadata = data['metadata']

            ################################ ABSTRACT ###################################################
            abstract_text = paper_metadata['abstractText'].translate(punct_removal_table).lower()
            #1. BERT et al encoding
            input_ids = torch.tensor([tokenizer.encode(abstract_text)]).unsqueeze(0) 
            outputs = model(input_ids)
            last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
            file_dict['attention_based_encoding'] = last_hidden_states[0][0]
            #file_dict['feature_extraction_encoding'] = nlp(abstract_text)[0][0] #DistilBERT
            #2. TFIDF ENCODING
            file_dict['tfidf_encoding'] = tfidf_matrix.toarray()[file_number]
            file_number = file_number + 1
            ################################ ABSTRACT ###################################################

            ################################ TITLE ###################################################
            title = paper_metadata['title'].lower()
            with open(os.path.join(data_path,'WordsFromTitleofTop200Papers.txt'),encoding="utf8", 'r') as f:
                top_200_titles_words_counter = Counter([word for word in f.read().translate(punct_removal_table).lower().split() if word not in stop_words])
                #TAKE TOP 5%
                top_200_titles_vocab = [key for key,value in top_200_titles_words_counter.most_common(int(0.05*len(top_200_titles_words_counter))).items()]
            #1. IF TITLE CONTAINS ATLEAST 2 WORDS
            if set(title.split()).intersection(set(top_200_titles_vocab)) > 0:
                file_dict['words_from_top_200_title'] = True
            #2. TITLE LENTGTH
            file_dict['title_length'] = textstat.lexicon_count(title, removepunct=True)
            #3. TITLE COMPLEXITY
            flesch = 1/textstat.flesch_reading_ease(title)
            dale_chall = textstat.dale_chall_readability_score(title)
            file_dict['title_complexity'] = (flesch + dale_chall)/2
            ################################ TITLE ###################################################

            ################################ AUTHORS ###################################################
            #1. NUMBER OF AUTHORS
            file_dict['number_of_authors'] = paper_metadata['authors']
            
            #2. AUTHOR AFFILIATION
            author_emails = paper_metadata['emails']
            author_institutes = [email.split('@')[1] for email in author_emails]
            research_strength_score = 0
            for institute in author_institutes:
              closest_matches = difflib.get_close_matches(institute, email_institute_affiliation_mapper.keys())
              if len(closest_matches) == 0:
                #if tthere are no closest matches it's not from a university from our list, 
                #which means its either a strange university or a corporate company. 
                #either way its safe to give it a HIGH SCORE - even if the paper is not from a good source
                #other parameters will take care of it 
                research_strength_score = research_strength_score + 60 #DECISION!
              else
                affiliated_unis_from_string_match = [email_institute_affiliation_mapper[match] for match in closest_matches]
                for affiliated_uni in affiliated_unis_from_string_match:
                  try:
                    sub_score = sub_score + university_score_df.loc[university_score_df.institute ==  affiliated_uni, 'count'].values[0]
                  except Exception as e:
                    #if it's an excpetion this is a university alright, 
                    #but doesnt fall in our 100 unis of csraniking scores
                    #this means its mostly not a top uni, so it is safe to give it a LOW SCORE
                    sub_score = sub_score + 6 #DECISION!
                closest_matches_avg_score = sub_score/len(affiliated_unis_from_string_match)
                research_strength_score = research_strength_score + closest_matches_avg_score
            file_dict['research_strength_score'] = research_strength_score/len(author_institutes)                    
            ################################ AUTHORS ###################################################

            ################################ REFERENCES ###################################################
            references_list,ref_mentions_list = paper_metadata['references'],paper_metadata['referenceMentions']
            #1. NUM OF REFRERENCES
            file_dict['num_of_references'] = len(references_list)
            #2. MOST RECENT REFERENCE YEAR
            ref_years_list = [ref_dict['year'] for ref_dict in references_list]
            file_dict['most_recent_ref_year'] = max(ref_years_list)
            #3. AVG LENGTH OF REF MENTION
            file_dict['avg_len_of_ref_mention'] = statistics.mean([ref_dict['endOffset'] - ref_dict['startOffset'] for ref_dict in ref_mentions_list])
            #4. NUMBER OF RECENT REFERENCES (current recent ref behnchmark = 4)
            file_dict['num_of_recent_references'] = sum([1 for year in ref_years_list if paper_metadata['year']-year<4])
            ################################ REFERENCES ###################################################
                
            ################################ CONTENT ###################################################
            #content housekeeping
            sections = paper_metadata['sections']
            section_content = ''
            for section in sections:
                section_content = section_content + " " + section['text'].translate(punct_removal_table).lower()
            file_dict['contains_githib_link'],file_dict['contains_appendix'] = False,False
            
            #1. NUMBER OF SECTIONS
            file_dict['number_of_sections'] = len(sections)
            #2. CONTAINS GITHUB LINK
            for section in sections:
                if section['text'].lower().contains('github'):
                    file_dict['contains_githib_link'] = True
                    break
            #3. READABILITY
            flesch_score,dale_chall_score = 0,0
            for section in sections:
                flesch_score = flesch_score + textstat.flesch_reading_ease(section['text'])
                dale_chall_score = dale_chall_score + textstat.dale_chall_readability_score(section['text'])
            flesch_score,dale_chall_score = flesch_score/file_dict['number_of_sections'],dale_chall_score/file_dict['number_of_sections']  
            file_dict['content_complexity'] = ((1/flesch) + dale_chall)/2
            #4. CONTAINS APPENDIX
            for section in sections:
                if section['heading'].contains('APPENDIX') or section['heading'].split()[0] in set(string.ascii_uppercase):
                    file_dict['contains_appendix'] = True
                    break
            #5. NUMBER OF UNIQUE WORDS
            file_dict['number_of_unique_words'] = len(Counter(section_content))
            ################################ CONTENT ###################################################
            
    return paper_data_df

list

In [0]:
def build_affiliation_dictionary(author_university_df,university_score_df):

  interim_mapper,mapper = {},{}
  for index, row in author_university_df.iterrows():
      # name,affiliation,homepage,scholarid
      link = row['homepage']
      try:
          if "http" in link:
              base = link.split("/")[2]
          else:
              base = link.split("/")[0]
      
          if "www" in base:
              base = ".".join(base.split(".")[1:])
              
          if base in interim_mapper:
              if not row["affiliation"] in interim_mapper[base]:
                  interim_mapper[base][row["affiliation"]] = 1
              else:
                  interim_mapper[base][row["affiliation"]] += 1
          else:
              interim_mapper[base] = {}
              interim_mapper[base][row["affiliation"]] = 1
              
      except Exception as e:
        print("Homepage : ",row['homepage']," defaulted!")

  # for base in interim_mapper:
  #     if len( interim_mapper[base].keys()) > 1:
  #         print(base, interim_mapper[base])

  #interim_mapper now has the email ids, and number of times, they were refferred to as an affiliations.
  #we ignore if number of affliations are more than 4 as then they are generic ids like gmail.com and shoudn't be mapped
  #else we take the maximum

  for base in interim_mapper:
      if len(interim_mapper[base].keys()) < 2:
          #calculate the maximum referred affiliation
          max_count = 0
          max_ff = ""
          for aff in interim_mapper[base]:
              if interim_mapper[base][aff] > max_count:
                  max_count = interim_mapper[base][aff]
                  max_ff = aff
          mapper[base] = aff
    
  with open("processed_data/affiliation_dict", "wb") as output_file:
      pickle.dump(mapper, output_file)

  return mapper

In [0]:
def build_and_save_tfidf_model():

  #collect corpus
  corpus = []
  directory_in_string = data_path + '/iclr_2017/train/parsed_pdfs'
  directory_content = os.fsencode(directory_in_string)
  for file in os.listdir(directory_content):
      filename = os.fsdecode(file)
      with open(os.path.join(directory_in_string, filename),encoding="utf8") as file:
          paper_metadata = json.load(file)['metadata']
          abstract_text = paper_metadata['abstractText'].translate(punct_removal_table)
          corpus.append(abstract_text)

  vectorizer = TfidfVectorizer(analyzer='word', stop_words = stop_words)
  X = vectorizer.fit_transform(corpus)
  return X

In [47]:
'''model_class, tokenizer_class, pretrained_weights = MODELS['DistilBertModel']
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

# Encode text
input_ids = torch.tensor([tokenizer.encode("Empirically, it is important observation of  all mankind that makes it necessary to incorporate a concept like recursion", add_special_tokens=True)])
outputs = model(input_ids)
last_hidden_states = outputs[0]
print(last_hidden_states.tolist()[0][0][345])'''

0.06255143880844116
